Prueba de cálculo de salario promedio general  y desagregado por sexo en la Ciudad de México

Serie longitudinal: últimos trimestres de 2021-2024

In [17]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, janitor, stringr, writexl)

Descargar y leer los archivos csv

In [18]:
dir_datos <- 'enoe_datos'
if (!dir.exists(dir_datos)) {
    dir.create(dir_datos)
}

In [19]:
descargar_enoe_csv <- function() {
    options(timeout = 90000)

    descargar_extraer <- function(url, destino) {
        temp <- tempfile()
        download.file(url, temp, mode = 'wb')
        unzip(temp, exdir = destino)
        unlink(temp)

    }
    
    #iterar los años de descarga    
    for(i in 21:22) {
        url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_20', i, '_trim4_csv.zip')
        descargar_extraer(url, dir_datos)
    }

    for(i in 23:24) {
        url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_20', i, '_trim4_csv.zip')
        descargar_extraer(url, dir_datos)
    }
}

In [20]:
descargar_enoe_csv()

In [21]:
archivos <- list.files(path = dir_datos, pattern = 'sdem.*\\.csv$', full.names = TRUE, ignore.case = TRUE)
datos <- archivos %>%
lapply(read_csv, show_col_types = FALSE)


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


In [22]:
tablas <- list()  # Crear lista vacía para almacenar los resultados

for (i in seq_along(datos)) {
  tablas[[i]] <- datos[[i]] %>%
    clean_names() %>%
    mutate(ano = as.numeric(str_extract(archivos[i], '\\d{2}(?=\\.csv$)')))
}

sdem <- bind_rows(tablas)


Diseño muestral

In [23]:
#Escenario complejo
enoe_svy <- as_survey_design(
    .data = sdem,
    ids = upm,
    strata = est_d_tri,
    weights = fac_tri,
    nest = TRUE 
) 

Cálculo de salario promedio

In [24]:
#Población general
gral_svy <- enoe_svy %>%
filter(r_def ==00, (c_res == 1 | c_res == 3),
         eda >= 15  & eda <= 98) %>%
filter(ent == 9, clase2 == 1) #Filtrar por población ocupada en la Ciudad de México

In [25]:
##Ocupados totales

gral_svy %>%
    group_by(ano) %>%
    summarize(ocupados = survey_total(vartype="cv"))

ano,ocupados,ocupados_cv
<dbl>,<dbl>,<dbl>
21,4465106,0.02862150
22,4698174,0.02704023
23,4657701,0.02614882
24,5010045,0.02470715


In [38]:
#Salario promedio general
salario_general <- gral_svy %>%
group_by(ano) %>%
summarise(sal_prom_gral = survey_mean(ingocup, na.rm = TRUE,vartype="cv"))
salario_general

ano,sal_prom_gral,sal_prom_gral_cv
<dbl>,<dbl>,<dbl>
21,4426.396,0.04723698
22,4177.289,0.04604882
23,4982.462,0.05547990
24,6182.284,0.05409411


In [45]:
#Salario promedio por sexo
salario_sex <- gral_svy %>%
group_by(ano, sex) %>%
summarise(sal_prom = survey_mean(ingocup, na.rm = TRUE, vartype="cv"))
salario_sex


ano,sex,sal_prom,sal_prom_cv
<dbl>,<dbl>,<dbl>,<dbl>
21,1,4976.284,0.06213587
21,2,3737.557,0.04918425
22,1,4481.992,0.05180844
22,2,3805.648,0.06257485
23,1,5245.048,0.06716809
23,2,4663.149,0.07953083
24,1,6586.656,0.07467182
24,2,5740.381,0.06102456


In [ ]:
salario_sex<-salario_sex%>%
select(-sal_prom_cv) %>%
pivot_wider(names_from = sex, values_from = sal_prom, names_prefix = "sal_prom_")
salario_sex 


ano,sal_prom_1,sal_prom_2
<dbl>,<dbl>,<dbl>
21,4976.284,3737.557
22,4481.992,3805.648
23,5245.048,4663.149
24,6586.656,5740.381


In [47]:
salario_general <- salario_general %>%
select(-sal_prom_gral_cv) %>%
right_join(salario_sex, by = 'ano') %>%
select(ano, sal_prom_gral, sal_prom_1, sal_prom_2) %>%
rename(sal_prom_hombres = sal_prom_1, sal_prom_mujeres = sal_prom_2)
salario_general

ano,sal_prom_gral,sal_prom_hombres,sal_prom_mujeres
<dbl>,<dbl>,<dbl>,<dbl>
21,4426.396,4976.284,3737.557
22,4177.289,4481.992,3805.648
23,4982.462,5245.048,4663.149
24,6182.284,6586.656,5740.381
